# Converting Local Data to NWB

In [1]:
%config Completer.use_jedi = False

In [2]:
import os
from os.path import join as pjoin

from datetime import datetime
from dateutil.tz import tzlocal

import h5py
import joblib
import numpy as np
import pandas as pd

from pynwb import NWBFile, NWBHDF5IO, TimeSeries
from pynwb.file import Subject
from pynwb.behavior import Position, SpatialSeries
from pynwb.ecephys import ElectricalSeries, SpikeEventSeries

### Settings

In [3]:
folder = 'data/'

In [4]:
# Load behavior data
df = pd.read_csv(pjoin(folder, 'behavior', 'events.csv'))

## Initialize a NWB File

Set up the file.

### Define Recording Metadata

In [5]:
# Define metadata for NWB file
session_description = 'XX'
experiment_description = 'Example conversion to NWB format.'
identifier = 'XX'
experimenter = 'XX'
lab = 'U01 Group'
institution = 'Columbia'
session_id = '001'

### Define Subject Information

In [6]:
# Set subject information
age = None
sex = None
description = None
species = 'human'
subject_id = None

In [7]:
# Create subject object
subject = Subject(age=age,
                  sex=sex,
                  description=description, 
                  species=species,
                  subject_id=subject_id)

### Collect together into NWB file

In [8]:
# Initialize a NWB file
nwbfile = NWBFile(session_description=session_description,
                  identifier=identifier, 
                  session_start_time=datetime.now(tzlocal()),
                  experimenter=experimenter,
                  lab=lab,
                  institution=institution,
                  experiment_description=experiment_description,
                  subject=subject,
                  session_id=session_id)

## Recording Definition

### Device(s)

In [9]:
# Device information
device_name = 'RECORDING DEVICE'
device_desc = 'RECORDING DEVICE DESCRIPTION'
device_manu = 'RECORDING DEVICE MANUFACTURER'

# Create device object
device = nwbfile.create_device(device_name, device_desc, device_manu)

In [10]:
# Check out the defined device
device

RECORDING DEVICE pynwb.device.Device at 0x140467618903952
Fields:
  description: RECORDING DEVICE DESCRIPTION
  manufacturer: RECORDING DEVICE MANUFACTURER

### Electrodes

In [11]:
# Add electrode description
location = 'WHERE'
electrode_name = '{}-microwires-{}'.format('A', 'chnum')
description = "Behnke Fried/Micro Inner Wire Bundle....ADD DETAILS."

# Add electrode group
electrode_group = nwbfile.create_electrode_group(electrode_name,
                                                 description=description,
                                                 location=location,
                                                 device=device)

In [12]:
# Define / get electrode information
x_pos, y_pos, z_pos = 0.0, 0.0, 0.0
imp = np.nan
location = 'place'
filtering = '0, np.inf'
reference = None

In [13]:
# Add electrode to NWB
n_electrodes = 5
for ind in range(n_electrodes):
    nwbfile.add_electrode(x_pos, y_pos, z_pos, imp, location, filtering, electrode_group, 
                          id=ind, reference=reference)

## Stimuli

Chest coordinates define our stimuli of interest. 

Note that they are saved as strings within the DF, and so need type casting.

In [14]:
# Check the chest positions
chest_positions = set(df['chest_coords']).pop()
print(chest_positions)

[(63.90156, -38.97487), (103.6756, 53.94187)]


In [15]:
# Wrangle the string values of the chest coordinates
for char in ['[', ']', '(', ')']:
    chest_positions = chest_positions.replace(char, '')
chest_positions = chest_positions.split(', ')
chest_positions = [float(val) for val in chest_positions]
chest_positions = np.reshape(chest_positions, (2, 2))

In [16]:
# Create an NWB object to store the chest positions
chest_description = 'Positions of the chests.'
chest_position = TimeSeries(name='chest_positions',
                            data=chest_positions,
                            rate=0.,
                            unit='pos',
                            description=chest_description) 

# Add chest position definition to file
nwbfile.add_stimulus(chest_position)

# Note: could add this as two different stimuli, one for each chest, perhaps (?)

## Behaviour data

### Trial Data

In [17]:
nwbfile.add_trial_column('choice_point_time', 'Time when subject arives at the direction choice point.')
nwbfile.add_trial_column('button_press_time', 'Time when subject presses the button.')

In [18]:
times = np.array(df['spikeoffset'])

In [19]:
# Get all trial start times
trial_start_times = times[np.hstack([np.array([0]), np.where(np.diff(df['trial']))[0]])]

# Get button presses, and fill first half of session, with not button presses
button_times = np.hstack([np.array([np.nan] * 18),
                          times[np.where(df['button_press'])[0]]])

# Add a couple NaNs to the beginning (no choice points in first two trials?)
choice_point_times = np.hstack([np.array([np.nan, np.nan]),
                                times[np.where(df['choice_point'])[0]]])

In [20]:
n_trials = len(trial_start_times) - 1

In [21]:
# Add event information to NWB file
for t_ind in range(n_trials):
    nwbfile.add_trial(start_time=trial_start_times[t_ind],
                      stop_time=trial_start_times[t_ind + 1] -1,
                      choice_point_time=choice_point_times[t_ind],
                      button_press_time=button_times[ind])

### Position Data

In [22]:
# Grab location data
times = np.array(df['spikeoffset'])
loc_data = np.vstack([df['x'], df['z']])

In [23]:
position = Position(name='position')

In [24]:
position.create_spatial_series(name = 'xy_pos',
                               data = loc_data,
                               timestamps=times,
                               reference_frame='middle',
                               description='XY position....')

xy_pos pynwb.behavior.SpatialSeries at 0x140467607049552
Fields:
  comments: no comments
  conversion: 1.0
  data: [[142.9      142.8993   142.8983   ... 151.0366   150.8509   142.9     ]
 [  8.9        8.900005   8.900008 ...   9.314151   9.257134   8.9     ]]
  description: XY position....
  interval: 1
  reference_frame: middle
  resolution: -1.0
  timestamps: [1660369.0700531  1670594.83218193 1670610.83197594 ... 3562898.81801605
 3562924.8175621  3563926.79405212]
  timestamps_unit: seconds
  unit: meters

In [25]:
# position.create_spatial_series(name='speed',
#                                data = np.array(df['speed']),
#                                timestamps=times,
#                                reference_frame='0')

# position.create_spatial_series(name='linear_position',
#                                data = np.array(df['linear_position']),
#                                timestamps=times,
#                                reference_frame='0')

In [26]:
# Add spatial series to NWB file
nwbfile.add_acquisition(position)

#### Add position derived measures as ProcessingModule

In [27]:
from pynwb import ProcessingModule

In [28]:
speed = TimeSeries(name='speed',
                   data = np.array(df['speed']),
                   unit = 'm/s', # ??
                   timestamps=times)

linear_position = TimeSeries(name='linear_position',
                             data = np.array(df['linear_position']),
                             unit = 'm', # ??
                             timestamps=times)

In [29]:
position_things = ProcessingModule(name='position_measures',
                                   description='Derived measures related to position data.',
                                   data_interfaces=[speed, linear_position])

In [30]:
nwbfile.add_processing_module(position_things)

position_measures pynwb.base.ProcessingModule at 0x140467607049648
Fields:
  data_interfaces: {
    linear_position <class 'pynwb.base.TimeSeries'>,
    speed <class 'pynwb.base.TimeSeries'>
  }
  description: Derived measures related to position data.

## Spiking Data

In [31]:
# Get a list of the available spike files
spike_files = os.listdir(pjoin(folder, 'spiketimes'))

In [32]:
# Specify additional metadata columns for units
#nwbfile.add_unit_column('location', 'The anatomical location of this unit.')

In [33]:
# Add each unit to the NWB file
for ind, spike_file in enumerate(spike_files):
    
    # Load spike file & get spike data
    with h5py.File(pjoin(folder, 'spiketimes', spike_file), 'r') as h5file:
        spike_data = h5file['spike_data_sorted']
    
        # Add unit data
        nwbfile.add_unit(id=ind,
                         electrodes=[0],
                         waveform_mean=np.mean(spike_data['spikeWaveforms'][:], 0),
                         spike_times=spike_data['spikeTimes'][:])
        
        # Add spike sorting extra information (?)
        # this is where could add 'cluster' and 'class' as a processing element
        #cluster=spike_data['spikeClusters'][:],
        #classes=spike_data['spikeClasses'][:])

## Field Data

In [34]:
# Field metadata (FIX WITH REAL VALUES)
srate = 1000. # ??
field_start_time = 0.

# NOTE - CAN ALTERNATELY SPECIFY LFP TIMESTAMPS
#lfp_times = np.arange(0., len(ephys_data))

# XXX
electrode_table_region = nwbfile.create_electrode_table_region([0], 'xx')

In [35]:
lfp_files = os.listdir(pjoin(folder, 'micro_lfp'))

In [36]:
# # Add each LFP trace as a new object
# for ind, lfp_file in enumerate(lfp_files):

#     with open(pjoin(folder, 'micro_lfp', lfp_files[0]), 'rb') as pfile:
#         ephys_data = joblib.load(pfile)
        
#         # Initialize 
#         ephys_ts = ElectricalSeries('field_data_' + str(ind),
#                                     ephys_data,
#                                     electrode_table_region,
#                                     starting_time=field_start_time,
#                                     rate=srate,
#                                     resolution=np.inf,
#                                     comments="...",
#                                     description="...")
        
#         # Add LFP data
#         nwbfile.add_acquisition(ephys_ts)

In [37]:
# Add LFP data as a combined array for all channels
for ind, lfp_file in enumerate(lfp_files):

    lfps = []
    with open(pjoin(folder, 'micro_lfp', lfp_files[0]), 'rb') as pfile:
        lfps.append(joblib.load(pfile))
        
ephys_data = np.array(lfps)

# Add LFP data to NWB
ephys_ts = ElectricalSeries('field_data',
                            ephys_data,
                            electrode_table_region,
                            starting_time=field_start_time,
                            rate=srate,
                            resolution=np.inf,
                            comments="...",
                            description="...")

nwbfile.add_acquisition(ephys_ts)

### Save out local data file

In [38]:
# Save out an example NWB file
with NWBHDF5IO('nwb_local_data.nwb', 'w') as io:
    io.write(nwbfile)